### Setup (only required for the first run on the Spark cluster)

In [48]:
!pip install pandas

In [49]:
ROOT_PATH = 'gs://news_public_datasets4/adressa'
ROOT_PATH = 'C:\\Users\\operi157093\\Desktop\\Master\\First Year\\Semester B\\Recommendation Systems\\Assignments\\Project\\'
#TODO: Upload this file (generated by the ACR module training) to GCS before calling spark script
!gsutil cp {ROOT_PATH}/data_transformed/adressa_articles.csv .

'gsutil' is not recognized as an internal or external command,
operable program or batch file.


### Loading dependencies

In [50]:
import os
print(os.environ['SPARK_HOME'])
print(os.environ['JAVA_HOME'])
print(os.environ['PATH'])

C:\Spark\spark-3.0.0-bin-hadoop2.7
C:\Program Files\Java\jdk-11.0.7
C:\Users\operi157093\AppData\Local\Continuum\anaconda3;C:\Users\operi157093\AppData\Local\Continuum\anaconda3\Library\mingw-w64\bin;C:\Users\operi157093\AppData\Local\Continuum\anaconda3\Library\usr\bin;C:\Users\operi157093\AppData\Local\Continuum\anaconda3\Library\bin;C:\Users\operi157093\AppData\Local\Continuum\anaconda3\Scripts;C:\Users\operi157093\AppData\Local\Continuum\anaconda3\bin;C:\Users\operi157093\AppData\Local\Continuum\anaconda3\condabin;C:\Users\operi157093\AppData\Local\Continuum\anaconda3;C:\Users\operi157093\AppData\Local\Continuum\anaconda3\Library\mingw-w64\bin;C:\Users\operi157093\AppData\Local\Continuum\anaconda3\Library\usr\bin;C:\Users\operi157093\AppData\Local\Continuum\anaconda3\Library\bin;C:\Users\operi157093\AppData\Local\Continuum\anaconda3\Scripts;C:\Program Files (x86)\Common Files\Oracle\Java\javapath;C:\Program Files (x86)\RSA SecurID Token Common;C:\Program Files\RSA SecurID Token Com

In [51]:
import os
import json
import pandas as pd
import pickle
import datetime


import hashlib
import math
import matplotlib
%matplotlib inline

In [52]:
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType

## Loading articles pre-processed

In [53]:
articles_original_df = pd.read_csv(ROOT_PATH + 'adressa_articles.csv')

In [54]:
articles_original_df.columns

Index(['adressa-access', 'author_1st', 'category0', 'category1', 'category2',
       'concepts', 'created_at_ts', 'entities', 'id', 'keywords', 'locations',
       'persons', 'publishtime', 'site', 'text_highlights', 'url',
       'id_encoded', 'category0_encoded', 'category1_encoded',
       'author_encoded', 'keywords_encoded', 'concepts_encoded',
       'entities_encoded', 'locations_encoded', 'persons_encoded'],
      dtype='object')

In [55]:
articles_original_df['url'].nunique()

73309

In [56]:
valid_articles_urls_to_ids_dict = dict(articles_original_df[['url','id_encoded']].apply(lambda x: (x['url'], x['id_encoded']), axis=1).values)
len(valid_articles_urls_to_ids_dict)

73309

### Loading user interactions

In [ ]:
#INTERACTIONS_PATH = 'gs://news_public_datasets4/adressa/one_week/*'
#INTERACTIONS_PATH = 'three_month/20170101'
from pyspark.shell import spark

DAYS_TO_LOAD_INTERACTIONS=17
#interaction_json_files = [os.path.join(ROOT_PATH, 'three_month/201701{:02d}'.format(day)) for day in range(1, DAYS_TO_LOAD_INTERACTIONS)]
interaction_json_files = [os.path.join(ROOT_PATH, 'one_week.tar/one_week/20170106')]
print('Loading interaction files: {}'.format(interaction_json_files))

interactions_df = spark.read \
  .option("mode", "PERMISSIVE") \
  .json(interaction_json_files)

print("Done reading")

Loading interaction files: ['C:\\Users\\operi157093\\Desktop\\Master\\First Year\\Semester B\\Recommendation Systems\\Assignments\\Project\\one_week.tar/one_week/20170106']


In [ ]:
interactions_df.printSchema()

In [ ]:
interactions_df.count()

In [ ]:
#Retrives article id from its cannonical URL (because sometimes article ids in interactions do no match with articles tables, but cannonical URL do)
def get_article_id_encoded_from_url(canonical_url):
    if canonical_url in valid_articles_urls_to_ids_dict:
        return valid_articles_urls_to_ids_dict[canonical_url]    
    return None

get_article_id_encoded_from_url_udf = F.udf(get_article_id_encoded_from_url, pyspark.sql.types.IntegerType())

In [ ]:
#Filtering only interactions whose url/id is available in the articles table
interactions_article_id_encoded_df = interactions_df.withColumn('article_id', get_article_id_encoded_from_url_udf(interactions_df['canonicalUrl']))
interactions_filtered_df = interactions_article_id_encoded_df.filter(interactions_article_id_encoded_df['article_id'].isNull() == False).cache()

In [ ]:
#Valid interactions
interactions_filtered_df.count()

In [ ]:
#Distinct items count
interactions_filtered_df.select('article_id').distinct().count()

In [ ]:
first_timestamp_ts = interactions_filtered_df.select('time').agg(F.min('time')).collect()[0][0] * 1000
first_timestamp_ts

### Analyzing elapsed time since publishing

In [ ]:
#interactions_filtered_df.filter(interactions_filtered_df['time'].isNull()).count()
#0

In [ ]:
def get_timestamp_from_date_str(value):
    if value is not None:
        value = str(value)
          #return int(datetime.datetime.strptime(value, '%Y-%m-%dT%H:%M:%S.%fZ').timestamp())
        return int(datetime.datetime.strptime(value, '%Y-%m-%d %H:%M:%S').timestamp())
    return None

get_timestamp_from_date_str_udf = F.udf(get_timestamp_from_date_str, pyspark.sql.types.IntegerType())

In [ ]:
interactions_filtered_with_publish_ts_df = interactions_filtered_df.withColumn('publish_ts', get_timestamp_from_date_str_udf(interactions_filtered_df['publishtime']))
interactions_filtered_with_publish_ts_df = interactions_filtered_with_publish_ts_df.withColumn('elapsed_min_since_published', ((F.col('time') - F.col('publish_ts')) / 60).cast(pyspark.sql.types.IntegerType()))

In [ ]:
#interactions_filtered_with_publish_ts_df.select('publishtime','publish_ts', 'time', 'elapsed_min_since_published').show(100)

In [ ]:
%%time
interactions_filtered_with_publish_ts_df.approxQuantile("elapsed_min_since_published", [0.10, 0.25, 0.50, 0.75, 0.90], 0.01)
#[49.0, 108.0, 334.0, 1020.0, 4611.0]

In [ ]:
elapsed_min_since_published_df = interactions_filtered_with_publish_ts_df.select('elapsed_min_since_published').toPandas()
print(len(elapsed_min_since_published_df[pd.isnull(elapsed_min_since_published_df['elapsed_min_since_published'])]))
elapsed_min_since_published_df.describe()

In [ ]:

'''
elapsed_min_since_published
count	2.600818e+06
mean	6.438622e+04
std	5.051825e+05
min	-3.151590e+05
25%	9.400000e+01
50%	2.580000e+02
75%	8.370000e+02
max	8.608278e+06
'''

### Analyzing clicks by article distribution

In [ ]:
#clicks_by_article_count_df = interactions_filtered_df.groupBy('article_id').count()
#clicks_by_article_count_df.approxQuantile("count", [0.01, 0.10, 0.25, 0.50, 0.75, 0.90, 0.99], 0.01)
#[1.0, 1.0, 1.0, 1.0, 2.0, 6.0, 33581.0]

### Processing categorical features

In [ ]:
def get_categ_features_counts_dataframe(interactions_spark_df,column_name):
    df_pandas = interactions_spark_df.groupBy(column_name).count().toPandas().sort_values('count', ascending=False)
    return df_pandas

In [ ]:
PAD_TOKEN = '<PAD>'
UNFREQ_TOKEN = '<UNF>'

def get_encoder_for_values(values):
    encoder_values = [PAD_TOKEN, UNFREQ_TOKEN] + values
    encoder_ids = list(range(len(encoder_values)))
    encoder_dict = dict(zip(encoder_values, encoder_ids))
    return encoder_dict

def get_categ_features_encoder_dict(counts_df, min_freq=100):
    freq_values = counts_df[counts_df['count'] >= 100][counts_df.columns[0]].values.tolist()
    encoder_dict = get_encoder_for_values(freq_values)
    return encoder_dict

def encode_cat_feature(value, encoder_dict):
    if value in encoder_dict:
        return encoder_dict[value]
    else:
        return encoder_dict[UNFREQ_TOKEN]

In [ ]:
countries_df = get_categ_features_counts_dataframe(interactions_filtered_df, 'country')
len(countries_df)

In [ ]:
countries_encoder_dict = get_categ_features_encoder_dict(countries_df)
len(countries_encoder_dict)

In [ ]:
cities_df = get_categ_features_counts_dataframe(interactions_filtered_df, 'city')
len(cities_df)

In [ ]:
cities_encoder_dict = get_categ_features_encoder_dict(cities_df)
len(cities_encoder_dict)

In [ ]:
regions_df = get_categ_features_counts_dataframe(interactions_filtered_df, 'region')
len(regions_df)

In [ ]:
regions_encoder_dict = get_categ_features_encoder_dict(regions_df)
len(regions_encoder_dict)

In [ ]:
devices_df = get_categ_features_counts_dataframe(interactions_filtered_df, 'deviceType')
print(len(devices_df))
devices_df

In [ ]:
devices_encoder_dict = get_categ_features_encoder_dict(devices_df)
len(devices_encoder_dict)

In [ ]:
os_df = get_categ_features_counts_dataframe(interactions_filtered_df, 'os')
print(len(os_df))
os_df

In [ ]:
os_encoder_dict = get_categ_features_encoder_dict(os_df)
len(os_encoder_dict)

In [ ]:
referrer_class_df = get_categ_features_counts_dataframe(interactions_filtered_df, 'referrerHostClass')
print(len(referrer_class_df))
referrer_class_df

In [ ]:
referrer_class_encoder_dict = get_categ_features_encoder_dict(referrer_class_df)
len(referrer_class_encoder_dict)

In [ ]:
encoders_dict = {
    'city': cities_encoder_dict,
    'region': regions_encoder_dict,
    'country': countries_encoder_dict,
    'os': os_encoder_dict,
    'device': devices_encoder_dict,
    'referrer_class': referrer_class_encoder_dict
}

### Processing numeric features

In [ ]:
%%time
active_time_quantiles = interactions_filtered_df.approxQuantile("activeTime", [0.10, 0.25, 0.50, 0.75, 0.90], 0.01)
print(active_time_quantiles)

In [ ]:
active_time_stats_df = interactions_filtered_df.describe('activeTime').toPandas()
active_time_stats_df

In [ ]:
active_time_mean = float(active_time_stats_df[active_time_stats_df['summary'] == 'mean']['activeTime'].values[0])
active_time_stddev = float(active_time_stats_df[active_time_stats_df['summary'] == 'stddev']['activeTime'].values[0])

### Splitting sessions

In [ ]:
'''
schema = T.StructType([
    T.StructField("userId", T.StringType()),
    T.StructField("min_ts", T.IntegerType())
])

@pandas_udf(schema, functionType=PandasUDFType.GROUPED_MAP)
def split_sessions(df):

    result_df = df[['userId']]
    result_df['min_ts'] = df['time'].min()
    
    return result

%%time
tmp = interactions_filtered_df.groupBy('userId').apply(split_sessions)
tmp.show(100)
'''

In [ ]:
def hash_str_to_int(encoded_bytes_text, digits):
    return int(str(int(hashlib.md5(encoded_bytes_text).hexdigest()[:8], 16))[:digits])      

In [ ]:
MAX_SESSION_IDLE_TIME_MS = 30 * 60 * 1000    #30 min

def close_session(session):
    size = len(session)
    
    #Creating and artificial session id based on the first click timestamp and a hash of user id
    first_click = session[0]
    session_id = (int(first_click['timestamp']) * 100) + hash_str_to_int(first_click['user_id'].encode(), 3)
    session_hour = int((first_click['timestamp'] - first_timestamp_ts) / (1000 * 60 * 60)) #Converting timestamp to hours since first timestamp
    
    #Converting to Spark DataFrame Rows, to convert RDD back to DataFrame
    clicks = list([T.Row(**click) for click in session])
    session_dict = {'session_id': session_id,
                    'session_hour': session_hour,
                    'session_size': size,
                    'session_start': first_click['timestamp'],
                    'user_id': first_click['user_id'],
                    'clicks': clicks 
                   }
    session_row = T.Row(**session_dict)
    
    return session_row
        
def transform_interaction(interaction):        
    return {
            'article_id': interaction['article_id'],
            'url': interaction['canonicalUrl'],
            'user_id': interaction['userId'],
            'timestamp': interaction['time'] * 1000, #converting to timestamp
            'active_time_secs': interaction['activeTime'],
            'country': encode_cat_feature(interaction['country'], encoders_dict['country']),
            'region': encode_cat_feature(interaction['region'], encoders_dict['region']),
            'city': encode_cat_feature(interaction['city'], encoders_dict['city']),
            'os': encode_cat_feature(interaction['os'], encoders_dict['os']),
            'device': encode_cat_feature(interaction['deviceType'], encoders_dict['device']),
            'referrer_class': encode_cat_feature(interaction['referrerHostClass'], encoders_dict['referrer_class']),
           }

def split_sessions(group):
    user, interactions = group
    #Ensuring items are sorted by time
    interactions_sorted_by_time = sorted(interactions, key=lambda x: x['time'])
    #Transforming interactions
    interactions_transformed = list(map(transform_interaction, interactions_sorted_by_time))

    
    sessions = []
    session = []        
    first_timestamp = interactions_transformed[0]['timestamp']
    last_timestamp = first_timestamp    
    for interaction in interactions_transformed:
        
        delta_ms = (interaction['timestamp'] - last_timestamp)
        interaction['_elapsed_ms_since_last_click'] = delta_ms 

        if delta_ms <= MAX_SESSION_IDLE_TIME_MS:    
            #Ignoring repeated items in session
            if len(list(filter(lambda x: x['article_id'] == interaction['article_id'], session))) == 0:        
                session.append(interaction)            
        else:
            #If session have at least 2 clicks (minimum for next click predicition)
            if len(session) >= 2:
                session_row = close_session(session)
                sessions.append(session_row)                
            session = [interaction]

        last_timestamp = interaction['timestamp']
            
    if len(session) >= 2:
        session_row = close_session(session)
        sessions.append(session_row)
        
    #if len(sessions) > 1:
    #    raise Exception('USER with more than one session: {}'.format(user))
    
    return list(zip(map(lambda x: x['session_id'], sessions), 
                    sessions))

In [ ]:
'''
#To debug
%%time
sessions_rdd = interactions_filtered_df.limit(1000).rdd.map(lambda x: (x['userId'], x)).groupByKey() \
                    .collect()

for row in sessions_rdd:
    print(split_sessions(row))
    print()
'''

In [ ]:
%%time
sessions_rdd = interactions_filtered_df.rdd.map(lambda x: (x['userId'], x)).groupByKey() \
                            .flatMap(split_sessions) \
                            .sortByKey() \
                            .map(lambda x: x[1])

#### Exporting sessions to JSON lines

In [ ]:
sessions_sdf = sessions_rdd.toDF()

In [ ]:
%%time
sessions_sdf.write.partitionBy("session_hour").json(os.path.join(ROOT_PATH,"data_transformed/sessions_processed_by_spark/"))

In [ ]:
sessions_sdf.count()

In [ ]:
def serialize(filename, obj):
    with open(filename, 'wb') as handle:
        pickle.dump(obj, handle)#, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
NAR_ENCODERS_PATH  = 'nar_encoders_adressa.pickle'
serialize(NAR_ENCODERS_PATH, encoders_dict)

In [ ]:
!gsutil cp {NAR_ENCODERS_PATH} {ROOT_PATH}/data_transformed/pickles/